## Download and extract the dataset
---
### Download link expires on October 24, 2019

In [0]:
!rm -r *
!wget https://transfer.sh/6BvD3/dataset.zip
!unzip -q dataset.zip
!ls -l --block-size=M

--2019-12-05 17:33:11--  https://transfer.sh/6BvD3/dataset.zip
Resolving transfer.sh (transfer.sh)... 144.76.136.153
Connecting to transfer.sh (transfer.sh)|144.76.136.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 454498692 (433M) [application/x-zip-compressed]
Saving to: ‘dataset.zip’

dataset.zip         100%[===================>] 433.44M  10.0MB/s    in 60s     

2019-12-05 17:34:25 (7.26 MB/s) - ‘dataset.zip’ saved [454498692/454498692]

warning [dataset.zip]:  76 extra bytes at beginning or within zipfile
  (attempting to process anyway)
error [dataset.zip]:  reported length of central directory is
  -76 bytes too long (Atari STZip zipfile?  J.H.Holm ZIPSPLIT 1.1
  zipfile?).  Compensating...
error:  expected central file header signature not found (file #363102).
  (please check that you have transferred or created the zipfile in the
  appropriate BINARY mode and that you have compiled UnZip properly)
total 434M
drwxr-xr-x 5 root root   1M Dec  5 1

In [0]:
working_dir =   "/content"
train_dir   =   "/content/dataset/train/"
val_dir     =   "/content/dataset/val/"
test_dir    =   "/content/dataset/test/"

# আসল খেলা

In [0]:
from keras.callbacks import *
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16

Using TensorFlow backend.


In [0]:
filepath = working_dir + "/epochs:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
csv_logger = CSVLogger('training.log')

callbacks_list = [checkpoint, csv_logger]

# Transfer Learning VGG16


In [0]:
# vgg = VGG16(include_top=False, weights='imagenet', input_shape=(32,32, 3))
# for layer in vgg.layers[:]:
#     layer.trainable = False

# vgg = VGG16(include_top=False, input_shape=(32,32, 3))

# model = Sequential()

# model.add(vgg)

# model.add(Flatten())
# model.add(Dropout(0.25))
# model.add(Dense(122, activation='softmax'))

# sgd = optimizers.SGD(lr=0.1)

# model.compile(loss='categorical_crossentropy',
#               optimizer=sgd,
#               metrics=['accuracy'])

model = Sequential()

model.add(Conv2D(32, kernel_size = 3, activation='relu', input_shape = (28, 28, 1)))
model.add(Conv2D(32, kernel_size = 3, activation='relu'))
model.add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
# model.add(Dropout(0.4))

model.add(Conv2D(64, kernel_size = 3, activation='relu'))
model.add(Conv2D(64, kernel_size = 3, activation='relu'))
model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
# model.add(Dropout(0.4))

model.add(Conv2D(128, kernel_size = 4, activation='relu'))
model.add(Flatten())
# model.add(Dropout(0.4))
model.add(Dense(122, activation='softmax'))

# COMPILE WITH ADAM OPTIMIZER AND CROSS ENTROPY COST
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [0]:
batch_size = 32

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        color_mode='grayscale',
        target_size=(28, 28),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        val_dir,
        color_mode='grayscale',
        target_size=(28, 28),
        batch_size=batch_size,
        class_mode='categorical')

Found 290135 images belonging to 122 classes.
Found 36223 images belonging to 122 classes.


In [0]:
model.fit_generator(
        train_generator,
        epochs=10,
        validation_data=validation_generator,
        callbacks=callbacks_list,
        verbose = 1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/10





9067/9067 [==============================] - 245s 27ms/step - loss: 1.2760 - acc: 0.6685 - val_loss: 0.5929 - val_acc: 0.8360

Epoch 00001: val_acc improved from -inf to 0.83599, saving model to /content/epochs:001-val_acc:0.836.hdf5
Epoch 2/10
9067/9067 [==============================] - 225s 25ms/step - loss: 0.5883 - acc: 0.8405 - val_loss: 0.4683 - val_acc: 0.8740

Epoch 00002: val_acc improved from 0.83599 to 0.87400, saving model to /content/epochs:002-val_acc:0.874.hdf5
Epoch 3/10
9067/9067 [==============================] - 224s 25ms/step - loss: 0.4894 - acc: 0.8670 - val_loss: 0.4455 - val_acc: 0.8769

Epoch 00003: val_acc improved from 0.87400 to 0.87687, saving model to /content/epochs:003-val_acc:0.877.hdf5
Epoch 4/10
9067/9067 [==============================] - 225s 25ms/step - loss: 0.4442 - acc: 0.8788 - val_loss: 0.4051 - val_acc: 0.8905

Epoch 00004: val_ac

In [0]:
from google.colab import files
files.download('training.log') 